In [34]:
import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Add the directory containing your script to the sys.path list
script_directory = r'C:\Users\Megha\Desktop\Ironhack\Week_6\Day_2\Afternoon'
sys.path.append(script_directory)

# Now you can import the functions from your script
from functions import *

# Initialize SpotiPy with user credentials


from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


In [35]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))



In [36]:
songs = pd.read_csv(r'C:\Users\Megha\Desktop\Ironhack\Week_6\Day_3\Afternoon\clustered_songs.csv')
songs.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster
0,0.838,0.626,0,-4.214,1,0.0396,0.00211,0.000059,0.0221,0.108,97.994,163514,4,3
1,0.611,0.478,0,-5.839,1,0.1070,0.03500,0.000000,0.1070,0.138,172.076,258158,4,0
2,0.744,0.697,11,-5.063,0,0.1970,0.01060,0.000000,0.0842,0.767,149.280,247413,4,2
3,0.823,0.703,8,-2.691,1,0.2980,0.01610,0.000001,0.2940,0.260,178.043,194214,4,2
4,0.649,0.200,2,-19.410,1,0.0507,0.83900,0.000306,0.2060,0.866,150.043,160325,4,4


In [45]:

def recommend_song(df, model):
    
    # Get the song and artist name from user
    title = input('What is your favourite song?\n')
    #artist = input('What is the artist name?\n')
    query = "track:" + title 
    results = sp.search(q=query, limit=1)
    items = results['tracks']['items']

    # Check if the song was found on spotify
    if len(items) == 0:
        print("Unable to find song!")
        sample = "No song"
        
        url = None
        return sample, url

    # Get the first song
    item = items[0]
    # Get the song id
    song_id = item['id']
    
    # Get song features
    features = sp.audio_features((song_id))[0]
    
    # Create a dataframe of useful features of the song 
    dict_features = { key : [features[key]] for key in list(features.keys()) }
    features_df=pd.DataFrame(dict_features)
    features_df=features_df.drop(columns=["key", "mode", "duration_ms", "time_signature", "analysis_url", "id", "track_href", "type", "uri"], axis=1)

    # Scale features, loading model and getting the user_cluster
    scaler = StandardScaler()
    features_scaled_np = scaler.fit_transform(features_df)
    features_scaled_df = pd.DataFrame(features_scaled_np, columns=features_df.columns)
  
    # Initialize KMeans with the specified number of clusters
    kmeans = KMeans(n_clusters=3)
    kmeans.fit(features_scaled_df)
    user_cluster = model.predict(features_scaled_df)[0]

    # Check song id in hot songs
    is_in_hot_df = df[(df['ids'] == song_id) & (df['hot'] == 'H')]

    if len(is_in_hot_df) > 0:
        # If it is recommend a song from hot songs in the same cluster
        sample_df = df[(df['hot'] == 'H') & (df['clusters'] == user_cluster)].sample(ignore_index=True)
    else:
        # Else recommend a song from the not hot songs in the same cluster
        sample_df = df[(df['hot'] == 'N') & (df['clusters'] == user_cluster)].sample(ignore_index=True)

    url = get_url(sample_df['ids'].values[0])
    title = sample_df['title'].values[0]

    return title, url

In [46]:
model = pd.read_csv(r'C:\Users\Megha\Desktop\Ironhack\Week_6\Day_3\Afternoon\lab-clustering-songs\merged_data.csv')

In [47]:
model.head()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,id,uri,track_href,analysis_url,duration_ms,time_signature,genre,Songs,hot/not_hot,Artists
0,0,0.838,0.626,0,-4.214,1,0.0396,0.00211,0.000059,0.0221,...,['1E8S3CSv20KrKT0C1yAjkO'],spotify:track:5YZRZqEkrteLmxvseHnQ2K,https://api.spotify.com/v1/tracks/5YZRZqEkrteL...,https://api.spotify.com/v1/audio-analysis/5YZR...,163514,4,Rap,Live Television,N,NaN
1,1,0.611,0.478,0,-5.839,1,0.1070,0.03500,0.000000,0.1070,...,['1AH60dMaEfNjzDoUIFwIQi'],spotify:track:1AH60dMaEfNjzDoUIFwIQi,https://api.spotify.com/v1/tracks/1AH60dMaEfNj...,https://api.spotify.com/v1/audio-analysis/1AH6...,258158,4,Rap,Beast Mode (feat. PnB Rock & YoungBoy Never Br...,N,NaN
2,2,0.744,0.697,11,-5.063,0,0.1970,0.01060,0.000000,0.0842,...,['2VevYw2rnvupfV0gWhXKxA'],spotify:track:17husT8tMWaBHcQqUpYDZQ,https://api.spotify.com/v1/tracks/17husT8tMWaB...,https://api.spotify.com/v1/audio-analysis/17hu...,247413,4,Hiphop,Make It Rain,N,NaN
3,3,0.823,0.703,8,-2.691,1,0.2980,0.01610,0.000001,0.2940,...,['50I7FNN2rl2rWiNwGxS146'],spotify:track:50I7FNN2rl2rWiNwGxS146,https://api.spotify.com/v1/tracks/50I7FNN2rl2r...,https://api.spotify.com/v1/audio-analysis/50I7...,194214,4,Rap,No Opp Left Behind,N,NaN
4,4,0.649,0.200,2,-19.410,1,0.0507,0.83900,0.000306,0.2060,...,['6UObhqa3DmniQ0YTWg2AqW'],spotify:track:5zDPkwHEID3BErCrgPbBOw,https://api.spotify.com/v1/tracks/5zDPkwHEID3B...,https://api.spotify.com/v1/audio-analysis/5zDP...,160325,4,Dark Trap,Storyboard for a Pessimist,N,NaN


In [48]:
recommend_song(model,KMeans)

What is your favourite song?
pait the red
Unable to find song!


('No song', None)